In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook

# PHYS 395 - week 1

Matt Wiens - #301294492

This notebook will be organized similarly to the lab script, with major headings corresponding to the headings on the lab script.

The TA's name (Ignacio) will be shortened to "IC" whenever used.

## Setup

In [ ]:
# Set default plot size
plt.rcParams['figure.figsize'] = (10, 7)

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

# Session 1

# Introduction to Python and the Jupyter environment

With permission from IC, I've used the import style I'm used to instead of `%pylab notebook`.

In [ ]:
# Print "hello world"
print("hello world")

In [ ]:
# Print pi with numbers of significant digits
for i in [3, 8, 16]:
    # IC: is there a cleaner way of formatting than what I've done here?
    print("%.*f" % (i, math.pi))

# Defining functions and plotting

## Lennard Jones Potential

The Lennard-Jones potential models interactions between pairs of neutral atoms or molecules. One equation for the Lennard-Jones potential $V_{\text{LJ}}$ is

\begin{equation}
    V_{\text{LJ}} =
        4 \epsilon
        \left(
            \left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^6 
        \right),
\end{equation}

where $\epsilon$ is the depth of the potential well, $\sigma$ is the finite distance at which the inter-particle potential is zero, and $r$ is the distance between the particles.

Note that if we express the distance between particles $r$ in terms of $\sigma$ we can remove the $\sigma$ parameter above.

The first function below gives the LJ potential with $r$ and $\sigma$ having the same units of length. The second function gives the LJ potential where $r$ is expressed in terms of $\sigma$ (and hence does not requre $\sigma$ as an argument).

In [ ]:
def lj_potential_1(r: float, sigma: float, eps: float):
    """Computes the L-J potential.

    r and sigma must be expressed in the same units of length.
    """
    l = sigma / r

    return 4 * eps * (l ** 12 - l ** 6)


def lj_potential_2(r: float, eps: float):
    """Computes the L-J potential.

    r must be expressed in terms of sigma.
    """
    return lj_potential_1(r, 1, eps)

## Plotting the LJ potential

First we will plot with $\epsilon = 1$, then we will plot with multiple values of $\epsilon$.

In [ ]:
# Note: these values are expressed in terms of sigma
r_vals = np.linspace(0.99, 15, 500)

In [ ]:
fig = plt.plot(r_vals, [lj_potential_2(r, 1) for r in r_vals])

Now we'll plot with different values of $\epsilon$. We'll keep using the same $r$ values as above.

In [ ]:
eps_vals = range(1, 4)

In [ ]:
fig = plt.figure()

# Plot
for eps in eps_vals:
    plt.plot(r_vals, [lj_potential_2(r, eps) for r in r_vals])

# Legend
ax = fig.axes[0]
ax.legend([r"$\epsilon$ = %d" % eps for eps in eps_vals])

# Labels
ax.set_xlabel(r"$r$/$\sigma$")
ax.set_ylabel(r"V/$\epsilon$")

# Save the figure
fig.savefig('ljplot.png', bbox_inches='tight')

# Reading in data

## Trajectory analysis

The data provided has many snapshots of a droid's position (in metres). Snapshots were taken every second.

In [ ]:
# Read in CSV to a dataframe
relative_file_path = "droid_traj.csv"

df = pd.read_csv(relative_file_path, names=["x", "y"])

The data loaded in is contained in a Pandas dataframe which has 250 rows and 2 columns.

In [ ]:
num_rows, num_cols = df.shape

print("rows: %d; cols: %d" % (num_rows, num_cols))

## Plotting the trajectory

In [ ]:
ax = df.plot.scatter(x="x", y="y")

## Path length

We will now calculate the path length traversed for each second. The path lengths will be stored in an array, where the indices correspond to the time (0-indexed).

In [ ]:
path_lengths = np.zeros(num_rows)

for t in range(1, num_rows):
    path_lengths[t] = path_lengths[t - 1] + np.linalg.norm(df.iloc[t] - df.iloc[t - 1])

# Also create a function for this, as an alternative interface
def s(t: int):
    """Returns path length traversed at time t (0-indexed)."""
    return path_lengths[t]

The total path length is approximately 129 units.

In [ ]:
print(path_lengths[-1])

## Speed

We will now estimate the speed of the droid at each second (we will assume the droid starts at rest). Since the time step between each data point is a second, we can simply take the difference of the path lengths.

In [ ]:
speeds = np.zeros(num_rows)

for t in range(1, num_rows):
    speeds[t] = s(t) - s(t - 1)

# Again we'll define a function for this
def v(t: int):
    """Returns estimated speed at time t (0-indexed)."""
    return speeds[t]

## Tangential acceleration

We will now estimate the tangential acceleration of the droid at each second.

In [ ]:
accels = np.zeros(num_rows)

for t in range(1, num_rows):
    accels[t] = v(t) - v(t - 1)

def a(t: int):
    """Returns estimated acceleration at time t (0-indexed)."""
    return accels[t]

## Plotting path length, speed, and tangential acceleration

In [ ]:
# Create figure and axes
fig, (ax1, ax2, ax3) = plt.subplots(3, 1)
fig.set_size_inches(10, 15)
fig.subplots_adjust(hspace=0.25)

# Add titles
ax1.title.set_text("path length")
ax2.title.set_text("speed")
ax3.title.set_text("tangential acceleration")

# Add labels
for ax in (ax1, ax2, ax3):
    ax.set_xlabel(r"$t$ (s)")
    ax.xaxis.set_label_coords(0.5, -0.05)

ax1.set_ylabel("m")
ax2.set_ylabel("m/s")
ax3.set_ylabel(r"m/$\mathrm{s}^2$")

# Plot
ts = list(range(num_rows))
ax1.plot(ts, path_lengths)
ax2.plot(ts, speeds)
ax3.plot(ts, accels)

# Tighten x-axis limits
for ax in (ax1, ax2, ax3):
    ax.set_xlim(0, num_rows)

# Session 1 Homework

## 1. Trajectory coloured by tangential acceleration

Since I assumed the droid started at rest, I will only plot the trajectory starting from the position of the 2nd data point. This removes the extreme acceleration at the beginning, so that we have a greater range of colours for the remainder of the trajectory.

In [ ]:
# Trim the dataset
df_trimmed = df.drop(np.arange(0, 3))
df_trimmed["accel"] = accels[3:]

In [ ]:
# Plot
ax = plt.scatter(
    x=df_trimmed["x"],
    y=df_trimmed["y"],
    c=df_trimmed["accel"],
    cmap="viridis"
)

plt.colorbar(ax);

## 2. Centripedal acceleration

Not sure how to do this yet.